## Summarization Pipeline
> In this pipeline we are using BART for summarizing transcript chunks over 1000 or 800 and refining it using LLM


In [1]:
import os
import time
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
HF_TOKEN = os.getenv("HF_TOKEN")

In [2]:
from langchain_groq.chat_models import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains.summarize import load_summarize_chain
from langchain_core.documents import Document
from huggingface_hub import InferenceClient
from typing import List

llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama-3.3-70b-versatile", # llama-3.3-70b-versatile good for summarizing texts and providing good context
    temperature=0.0,
    max_retries=2
)

hf_client = InferenceClient( # Using facebook/bart-large-cnn
    provider="hf-inference",
    api_key=HF_TOKEN
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

refine_template = PromptTemplate(template="""You are an expert meeting analyst. Your task is to create a comprehensive meeting summary.

Existing summary (if any):
{existing_answer}

New information from the meeting:
{text}

Instructions:
- Combine the existing summary with the new information
- Create a concise yet comprehensive summary
- Focus on key decisions, action items, and important discussion points
- Maintain a professional, clear tone
- Organize information logically

Refined Summary""")



In [3]:
# Step 2: BART Summarization Function
def summarize_chunk_with_bart(chunk_text: str, max_length: int = 150, min_length: int = 50) -> str:
    """Summarize a single chunk using BART model via HuggingFace Inference."""
    try:
        summary = hf_client.summarization(
            chunk_text,
            model="facebook/bart-large-cnn",
        )
        return summary.summary_text if hasattr(summary, 'summary_text') else summary
    except Exception as e:
        print(f"Error summarizing chunk: {e}")
        return chunk_text[:200]  # Fallback to truncated text

# Step 3: Process all chunks with BART
def summarize_chunks_with_bart(chunks: List[str]) -> List[str]:
    """Summarize all chunks using BART."""
    bart_summaries = []
    
    for i, chunk in enumerate(chunks):
        print(f"Summarizing chunk {i+1}/{len(chunks)} with BART...")
        summary = summarize_chunk_with_bart(chunk)
        bart_summaries.append(summary)
        time.sleep(0.6)  # Rate limiting
    
    return bart_summaries



In [4]:
initial_template = """You are an expert meeting analyst. Summarize the following meeting content:

{text}

Create a concise, detailed summary focusing on:
- Key discussion points
- Decisions made
- Action items
- Important insights

Summary:"""

initial_prompt = PromptTemplate(
    template=initial_template,
    input_variables=["text"]
)

# Step 5: Main Pipeline Function
def summarize_meeting_transcript(transcript: str) -> dict:
    """
    Complete pipeline to summarize meeting transcript.
    
    Args:
        transcript: The full meeting transcript text
        
    Returns:
        dict containing:
            - final_summary: The comprehensive final summary
            - bart_summaries: Individual BART summaries of chunks
            - chunk_count: Number of chunks processed
    """
    print("Starting meeting transcript summarization pipeline...")
    
    # Split transcript into chunks
    print("\n[1/3] Splitting transcript into chunks...")
    chunks = text_splitter.split_text(transcript)
    print(f"Created {len(chunks)} chunks")
    
    # Summarize each chunk with BART
    print("\n[2/3] Summarizing chunks with BART...")
    bart_summaries = summarize_chunks_with_bart(chunks)
    
    # Combine BART summaries into Documents for LangChain
    print("\n[3/3] Combining summaries with LLM using refine chain...")
    bart_summary_docs = [Document(page_content=summary) for summary in bart_summaries]
    
    # Create refine chain
    refine_chain = load_summarize_chain(
        llm=llm,
        chain_type="refine",
        question_prompt=initial_prompt,
        refine_prompt=refine_template,
        return_intermediate_steps=False,
        verbose=True
    )
    
    # Generate final summary
    result = refine_chain.invoke({"input_documents": bart_summary_docs})
    final_summary = result["output_text"]
    
    print("\n✓ Pipeline complete!")
    
    return {
        "final_summary": final_summary,
        "bart_summaries": bart_summaries,
        "chunk_count": len(chunks)
    }

In [5]:
transcript = """Mock Meeting Transcript: Q3 Feature Alignment

Meeting Title: Feature Phoenix - Rollout Strategy and Dependencies Date: Tuesday, November 25, 2025 Attendees: Alhaan (Tech Lead), Sofia (Product Manager), David (Marketing Lead), Ms. Mern (Developer) Duration: 30 Minutes

00:00 Alhaan: Good afternoon, everyone. This meeting is to finalize the launch timeline for Feature Phoenix and confirm all marketing, tech, and documentation dependencies are aligned. Sofia, can you start with a quick overview of the final user story?

01:30 Sofia: Absolutely. Feature Phoenix is a new analytics dashboard allowing our premium users to track their usage efficiency in real-time. The core user story is complete; the Dev team is now integrating the final data pipeline endpoints. The success metric is 20% adoption in the first month.

03:00 Alhaan: Thanks. Ms. Mern, how is the pipeline integration looking? Any roadblocks now that the FastAPI auth is stabilized?

04:30 Ms. Mern: The FastAPI portion is solid. We've defined the three necessary endpoints: /api/efficiency/daily, /api/efficiency/weekly, and /api/efficiency/summary. I estimate we need three more full development days to complete robust unit testing and deploy to the Staging environment.

06:30 Alhaan: Okay, so we're looking at Staging deployment by the end of Friday, November 28th. David, does that give Marketing enough time to prepare for the launch on Tuesday, December 3rd?

08:00 David: Launching on December 3rd requires the final UI/UX screenshots and a confirmed feature name by end of day, November 29th, so we can schedule the email campaign and update the landing page. Our dependency is the final, stable Staging link for screenshot capture.

10:00 Alhaan: Understood. Ms. Mern, can we promise a stable, QA-approved Staging link by Saturday morning? We need to lock down those front-end assets.

11:00 Ms. Mern: Yes, I can commit to having the Staging environment stable and QA-ready by Friday evening, November 28th. I will notify David personally when it's live.

12:00 Sofia: Excellent. My team will start drafting the internal documentation and training material based on the current feature set. Alhaan, do you foresee any major risk in the data model scaling beyond initial rollout?

14:00 Alhaan: Low risk for V1. The complexity of the current algorithm is manageable for our projected load. We'll start architectural planning for V2 scaling next quarter.

15:00 Alhaan: Summary of Action Items: 1. Ms. Mern to provide stable Staging link to David by Nov 28th, 7 PM. 2. David to finalize all launch creative (screenshots, name) by Nov 29th. 3. Sofia to complete internal training docs by Dec 2nd. Launch date remains Dec 3rd.

16:00 Alhaan: David, please send out the official launch countdown. Sofia, let's sync on documentation access tomorrow. Thank you all.

16:15 Alhaan: Thanks, team. Let's make this a great day."""

In [6]:
result = summarize_meeting_transcript(transcript)
print(result["final_summary"])

Starting meeting transcript summarization pipeline...

[1/3] Splitting transcript into chunks...
Created 4 chunks

[2/3] Summarizing chunks with BART...
Summarizing chunk 1/4 with BART...
Error summarizing chunk: Server error '504 Gateway Time-out' for url 'https://router.huggingface.co/hf-inference/models/facebook/bart-large-cnn'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/504
Summarizing chunk 2/4 with BART...
Error summarizing chunk: Server error '504 Gateway Time-out' for url 'https://router.huggingface.co/hf-inference/models/facebook/bart-large-cnn'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/504
Summarizing chunk 3/4 with BART...


KeyboardInterrupt: 